# XGBoost AutoScaling Example

abalone_dataset1_train.csv はダウンロードします。

In [1]:
import boto3
import sagemaker
from sagemaker.estimator import Estimator

boto_session = boto3.session.Session()
region = boto_session.region_name

sagemaker_session = sagemaker.Session()
base_job_prefix = 'xgboost-example'
role = sagemaker.get_execution_role()

default_bucket = sagemaker_session.default_bucket()
s3_prefix = base_job_prefix

training_instance_type = 'ml.m5.xlarge'

## Download Data and Prepare Training Input in S3

In [2]:
!aws s3 cp s3://sagemaker-sample-files/datasets/tabular/uci_abalone/train_csv/abalone_dataset1_train.csv .

download: s3://sagemaker-sample-files/datasets/tabular/uci_abalone/train_csv/abalone_dataset1_train.csv to ./abalone_dataset1_train.csv


In [3]:
!aws s3 cp abalone_dataset1_train.csv s3://{default_bucket}/xgboost-regression/train.csv

upload: ./abalone_dataset1_train.csv to s3://sagemaker-ap-northeast-1-117053077017/xgboost-regression/train.csv


In [5]:
from sagemaker.inputs import TrainingInput
training_path = f's3://{default_bucket}/xgboost-regression/train.csv'
train_input = TrainingInput(training_path, content_type="text/csv")

## Retrieve XGBoost Image and Prepare Training Estimator W/ HyperParameters

In [6]:
model_path = f's3://{default_bucket}/{s3_prefix}/xgb_model'

image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1",
    py_version="py3",
    instance_type=training_instance_type,
)

xgb_train = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    output_path=model_path,
    sagemaker_session=sagemaker_session,
    role=role
)

xgb_train.set_hyperparameters(
    objective="reg:linear",
    num_round=50,
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.7,
    silent=0,
)

## Model Training

In [7]:
xgb_train.fit({'train': train_input})

2022-11-11 02:32:52 Starting - Starting the training job...
2022-11-11 02:33:16 Starting - Preparing the instances for trainingProfilerReport-1668133972: InProgress
.........
2022-11-11 02:34:46 Downloading - Downloading input data...
2022-11-11 02:35:17 Training - Training image download completed. Training in progress.
2022-11-11 02:35:17 Uploading - Uploading generated training modelINFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value reg:linear to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[02:35:09] 2923x8 matrix with 23384 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
INFO:root:Single node trainin

## Retrieve Model Data

In [8]:
model_artifacts = xgb_train.model_data
model_artifacts

's3://sagemaker-ap-northeast-1-117053077017/xgboost-example/xgb_model/sagemaker-xgboost-2022-11-11-02-32-52-057/output/model.tar.gz'

## Create SM Client to Create Model, EP Config, EP

In [9]:
sm_client = boto3.client(service_name='sagemaker')

## Model Creation

In [10]:
from time import gmtime, strftime
model_name = 'xgboost-reg' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Model name: ' + model_name)

reference_container = {
    "Image": image_uri,
    "ModelDataUrl": model_artifacts
}

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer= reference_container)

print("Model Arn: " + create_model_response['ModelArn'])

Model name: xgboost-reg2022-11-11-02-36-04
Model Arn: arn:aws:sagemaker:ap-northeast-1:117053077017:model/xgboost-reg2022-11-11-02-36-04


## Endpoint Config Creation

In [11]:
endpoint_config_name = 'xgboost-config' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
#instance_type='ml.c5d.18xlarge'
#instance_type='ml.r6g.12xlarge'
instance_type='ml.m5.large'
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': instance_type,
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName': model_name,
        'VariantName': 'AllTraffic',
        }])

print("Endpoint config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

Endpoint config name: xgboost-config2022-11-11-02-36-05
Endpoint config Arn: arn:aws:sagemaker:ap-northeast-1:117053077017:endpoint-config/xgboost-config2022-11-11-02-36-05


## Endpoint Creation

In [12]:
%%time

import time

endpoint_name = 'xgboost-reg' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint name: ' + endpoint_name)

Endpoint name: xgboost-reg2022-11-11-02-36-05
CPU times: user 723 µs, sys: 112 µs, total: 835 µs
Wall time: 1.61 ms


In [13]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

Endpoint Arn: arn:aws:sagemaker:ap-northeast-1:117053077017:endpoint/xgboost-reg2022-11-11-02-36-05


In [14]:

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Endpoint Status: " + status)

print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

Endpoint Status: Creating
Waiting for xgboost-reg2022-11-11-02-36-05 endpoint to be in service...


## Sample Invocation

In [16]:
import boto3
smr = boto3.client('sagemaker-runtime')

resp = smr.invoke_endpoint(EndpointName=endpoint_name, Body=b'.345,0.224414,.131102,0.042329,.279923,-0.110329,-0.099358,0.0', 
                           ContentType='text/csv')

print(resp['Body'].read())

b'4.566554546356201'


## AutoScaling SageMaker Real-Time Endpoint

Here we define a scaling policy based off of invocations per instance. We set the maximum instance count to 4. We can define this using the Boto3 SDK. There's different types of scaling policies: Simple Scaling, Target Tracking Scaling, Step Scaling, Scheduled Scaling, and On-Demand Scaling. For this we'll be using Target Tracking Scaling and be using the Invocations Per Instance Metric as the basis for scaling.

Instance min - max が、1-4になっていることがコンソールで確認できます。

In [17]:
# AutoScaling client
asg = boto3.client('application-autoscaling')

# Resource type is variant and the unique identifier is the resource ID.
resource_id=f"endpoint/{endpoint_name}/variant/AllTraffic"

# scaling configuration
response = asg.register_scalable_target(
    ServiceNamespace='sagemaker', #
    ResourceId=resource_id,
    ScalableDimension='sagemaker:variant:DesiredInstanceCount', 
    MinCapacity=1,
    MaxCapacity=4
)

#Target Scaling
response = asg.put_scaling_policy(
    PolicyName=f'Request-ScalingPolicy-{endpoint_name}',
    ServiceNamespace='sagemaker',
    ResourceId=resource_id,
    ScalableDimension='sagemaker:variant:DesiredInstanceCount',
    PolicyType='TargetTrackingScaling',
    TargetTrackingScalingPolicyConfiguration={
        'TargetValue': 10.0, # Threshold
        'PredefinedMetricSpecification': {
            'PredefinedMetricType': 'SageMakerVariantInvocationsPerInstance',
        },
        'ScaleInCooldown': 300, # duration until scale in
        'ScaleOutCooldown': 60 # duration between scale out
    }
)

https://docs.aws.amazon.com/ja_jp/AWSCloudFormation/latest/UserGuide/aws-properties-applicationautoscaling-scalingpolicy-targettrackingscalingpolicyconfiguration.html

TargetValue

    The target value for the metric. Although this property accepts numbers of type Double, it won't accept values that are either too small or too large. Values must be in the range of -2^360 to 2^360. The value must be a valid number based on the choice of metric. For example, if the metric is CPU utilization, then the target value is a percent value that represents how much of the CPU can be used before scaling out. 

Instance min - max が、1-4になっていることがコンソールで確認できます。

## エンドポイントにリクエスト負荷をかける

In [21]:
#endpoint_name = 'xgboost-reg2022-11-11-01-57-49'
endpoint_name

'xgboost-reg2022-11-11-02-36-05'

In [22]:
#request_duration = 250
request_duration = 600
end_time = time.time() + request_duration
print(f"test will run for {request_duration} seconds")
i=0
while time.time() < end_time:
    resp = smr.invoke_endpoint(EndpointName=endpoint_name, Body=b'.345,0.224414,.131102,0.042329,.279923,-0.110329,-0.099358,0.0', 
                           ContentType='text/csv')
    i+=1
    #print(i)
print(i)

test will run for 600 seconds
25738


We can monitor these invocations through CloudWatch which you can access through the SageMaker console.

We can zoom in to monitor the InvocationsPerInstance metric more.

# モニタリングは別ノートブックで行います。

XGBoost-Abalone-monitoring.ipynbを開いてください。

In [ ]:
#response = sm_client.describe_endpoint(EndpointName=endpoint_name)
#status = response['EndpointStatus']
#print("Status: " + status)


#while status=='Updating':
#    time.sleep(1)
#    response = sm_client.describe_endpoint(EndpointName=endpoint_name)
#    status = response['EndpointStatus']
#    instance_count = response['ProductionVariants'][0]['CurrentInstanceCount']
#    print(f"Status: {status}")
#    print(f"Current Instance count: {instance_count}")